In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dropout, LSTM, Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from collections import Counter
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer

# define the model
def create_lstm_model(units, vocab_length, embedding_matrix, max_len, dropout):
    lstm_model = Sequential()
    embedding_layer = Embedding(vocab_length, 200, weights=[embedding_matrix], input_length=max_len, trainable=False)
    lstm_model.add(embedding_layer)
    lstm_model.add(Dropout(dropout))
    lstm_model.add(LSTM(units))
    lstm_model.add(Dense(5, activation='softmax'))
    lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    return lstm_model


# fix random seed for reproducibility
#tf.random.set_seed(7)

df = pd.read_csv('../../data/normalized_tweets.csv')
df = df[df['cyberbullying_type'] != 'other_cyberbullying']
# Reset index after filtering out the class
df.reset_index(drop=True, inplace=True)
#df["cyberbullying_type"].value_counts()

### try different length based on tweet lentgh
#df['text_len'] = [len(text.split()) for text in df.tweet_text]
#max_len = np.max(df['text_len'])
max_len = 50

X, y = df["tweet_text"], df["cyberbullying_type"]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform label encoder on the target variable
y = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#Each word in input used as a key, while a unique index is used as the value of the key 
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(x_train)

X_train = word_tokenizer.texts_to_sequences(x_train)
X_test = word_tokenizer.texts_to_sequences(x_test)

vocab_length = len(word_tokenizer.word_index) + 1

print(vocab_length)

####

X_train = pad_sequences(X_train, padding = 'pre', maxlen = max_len)
X_test = pad_sequences(X_test, padding = 'pre', maxlen = max_len)

# Load GloVe word embeddings and create a dictionary that willl contain words as keys, and their corresponging embedding list as values. 
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('../../glove_embeddings/glove.twitter.27B.200d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_length, 200)) ## change if the dimention of embedding changes above
i = 0
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
    else:
        i = i + 1
print(i)

35959
8246


In [10]:
X_train[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  4026, 16244,    56,     7,    14,     5,
           1,   304,   914,    14,    71], dtype=int32)

In [11]:
import warnings
warnings.filterwarnings("ignore")


model = KerasClassifier(model=create_lstm_model, units=128, batch_size=64, dropout=0.2, validation_split=0.2,optimizer__learning_rate=0.1, vocab_length=vocab_length, embedding_matrix=embedding_matrix, max_len=max_len)

# Define the grid search parameters
#param_grid = {
   # 'optimizer__learning_rate': [0.0001, 0.001, 0.1],
   # 'dropout': [None, 0.1, 0.2, 0.3, 0.5],
   # 'epochs': [5, 10],
   # 'units' : [32,64,128,256],
   # 'batch_size' : [32, 64, 128,256]
#}

param_grid = {
    'optimizer__learning_rate': [0.001, 0.005, 0.01],
    'dropout': [0.1, 0.5],
    'epochs': [10],
    'units' : [64,128],
    'batch_size' : [64]
}

# Define early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=0)
grid_result = grid.fit(X_train, y_train)


Epoch 1/10
256/256 [==============================] - 8s 30ms/step - loss: 0.4757 - acc: 0.8361 - val_loss: 0.2715 - val_acc: 0.9062
Epoch 2/10
256/256 [==============================] - 8s 31ms/step - loss: 0.2515 - acc: 0.9124 - val_loss: 0.2404 - val_acc: 0.9150
Epoch 3/10
256/256 [==============================] - 7s 26ms/step - loss: 0.2101 - acc: 0.9261 - val_loss: 0.2286 - val_acc: 0.9216
Epoch 4/10
256/256 [==============================] - 7s 26ms/step - loss: 0.1845 - acc: 0.9334 - val_loss: 0.2157 - val_acc: 0.9260
Epoch 5/10
256/256 [==============================] - 7s 27ms/step - loss: 0.1649 - acc: 0.9401 - val_loss: 0.2184 - val_acc: 0.9228
Epoch 6/10
256/256 [==============================] - 7s 28ms/step - loss: 0.1473 - acc: 0.9459 - val_loss: 0.2171 - val_acc: 0.9253
Epoch 7/10
256/256 [==============================] - 8s 30ms/step - loss: 0.1318 - acc: 0.9528 - val_loss: 0.2206 - val_acc: 0.9238
Epoch 8/10
256/256 [==============================] - 8s 31ms/step - 

KeyboardInterrupt: 

In [ ]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
  #  print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.929246 using {'batch_size': 64, 'dropout': 0.5, 'epochs': 10, 'optimizer__learning_rate': 0.001, 'units': 128}


In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.plot(lstm_model_history.history['acc'])
plt.plot(lstm_model_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(lstm_model_history.history['loss'])
plt.plot(lstm_model_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

NameError: name 'lstm_model_history' is not defined